In [4]:
pip install torchinfo torchviz graphviz seaborn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
from torchinfo import summary

# nếu file của bạn là model/cnn.py
from model.cnn import simpleCNN

# Khởi tạo model (33 lớp như trong code của bạn)
model = simpleCNN()

# In bảng tóm tắt với input 3x224x224
summary(
    model,
    input_size=(1, 3, 224, 224),
    col_names=("input_size", "output_size", "num_params", "kernel_size"),
    col_width=20,
    depth=4
)


Layer (type:depth-idx)                   Input Shape          Output Shape         Param #              Kernel Shape
simpleCNN                                [1, 3, 224, 224]     [1, 33]              --                   --
├─Conv2d: 1-1                            [1, 3, 224, 224]     [1, 64, 224, 224]    1,792                [3, 3]
├─ReLU: 1-2                              [1, 64, 224, 224]    [1, 64, 224, 224]    --                   --
├─MaxPool2d: 1-3                         [1, 64, 224, 224]    [1, 64, 112, 112]    --                   2
├─ConvReluBn: 1-4                        [1, 64, 112, 112]    [1, 128, 112, 112]   --                   --
│    └─Conv2d: 2-1                       [1, 64, 112, 112]    [1, 128, 112, 112]   73,728               [3, 3]
│    └─BatchNorm2d: 2-2                  [1, 128, 112, 112]   [1, 128, 112, 112]   256                  --
│    └─ReLU: 2-3                         [1, 128, 112, 112]   [1, 128, 112, 112]   --                   --
├─MaxPool2d: 1-5    

In [2]:
import torch
from torchviz import make_dot
from model.cnn import simpleCNN

# Luôn dùng CPU để dựng graph cho chắc (tránh lỗi CUDA khi render)
device = torch.device("cuda")

model = simpleCNN().to(device)
model.eval()

dummy = torch.randn(1, 3, 224, 224, device=device)
y = model(dummy)

# Vẽ đồ thị, include cả tham số để xem tên layer/weight
dot = make_dot(y, params=dict(model.named_parameters()))
dot.format = "png"                      # xuất ra PNG
dot.render("simplecnn_graph", cleanup=True)  # -> tạo file simplecnn_graph.png
print("Saved to simplecnn_graph.png")

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [ ]:
# --- core ---
import os, sys, numpy as np
import torch, torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

device = 'cuda' if torch.cuda.is_available() else 'cpu'
pin = torch.cuda.is_available()
print("Device:", device)

Device: cuda
